In [71]:
from lxml import html
import requests
from time import sleep
import random as rand

In [60]:
def scrape_definition_MW(word):
    """Scrapes a definition from Merriam-webster"""
    url = f"https://www.merriam-webster.com/dictionary/{word}"
    page = requests.get(url)
    
    xml_tree = html.fromstring(page.content)
    
    
    definition_arr = xml_tree.xpath(
        '//*[@id="dictionary-entry-1"]/div[2]/div[1]/span/div/span[2]/span/text()'
    )
    definition = "".join(definition_arr)
    entry = f"{word.upper()}: " + definition
    words = ''.join(list(filter(lambda x: x.isalpha() or x==' ', definition))).split(' ')
    return entry, words

def scrape_definition_wordnik(word):
    """Scrapes a definition from wordnik/duckduckgo"""
    word = word.lower() # To make sure it fits the URL scheme
    url = f"https://www.wordnik.com/words/{word}"
    page = requests.get(url)
    
    xml_tree = html.fromstring(page.content)
    
    # Create definition to add to dictionary
    for i in range(1, 100):
        definition_arr = xml_tree.xpath(
            f'//*[@id="define"]/div/ul[1]/li[{i}]/text()'
        )
        definition = "".join(definition_arr).strip()
        if len(definition) > 0:
            break
        
    entry = f"{word.upper()}: " + definition
    
    # Create next list of words to use
    words = ''.join(list(filter(lambda x: x.isalpha() or x==' ', definition))).split(' ')
    words = list(filter(lambda w: len(w) > 0 and all([c.isalpha() for c in w]), words))
    words = [w.lower() for w in words]
    return entry, words

scrape_definition_wordnik('and')

('AND: Connective: A word connecting a word, phrase, clause, or sentence with that which precedes it: a colorless particle without an exact synonym in English, but expressed approximately by ‘with, along with, together with, besides, also, moreover,’ the elements connected being grammatically coördinate.',
 ['connective',
  'a',
  'word',
  'connecting',
  'a',
  'word',
  'phrase',
  'clause',
  'or',
  'sentence',
  'with',
  'that',
  'which',
  'precedes',
  'it',
  'a',
  'colorless',
  'particle',
  'without',
  'an',
  'exact',
  'synonym',
  'in',
  'english',
  'but',
  'expressed',
  'approximately',
  'by',
  'with',
  'along',
  'with',
  'together',
  'with',
  'besides',
  'also',
  'moreover',
  'the',
  'elements',
  'connected',
  'being',
  'grammatically',
  'coördinate'])

In [78]:
super_dict = dict()

def main(seed="gold"):
    global super_dict
    
    # Begins with the seed word
    word = seed
    entry, words = scrape_definition_wordnik(word)
    super_dict_entry = {word.upper() : entry}
    super_dict.update(super_dict_entry)
    
    # Main loop to continuously add words to dictionary
    while 1:
        sleep(0.1)
        for i in range(len(words)):
            word = words[i].lower()
            # Check if given word is in dictionary already
            if word.upper() in super_dict.keys():
                print(f'"{word.upper()}" already added, skipping...')
                # In the event that all available words are already in the dictionary, try again with a random one
                if all([w.upper() in super_dict.keys() for w in words]):
                    main(seed=rand.choice(list(super_dict.keys()))) # Must convert to list first, rand.choice cant use dict_keys obj
                continue
            print(f'Querying "{word.upper()}"')
            entry, words = scrape_definition_wordnik(word)
            super_dict_entry = {word.upper() : entry}
            super_dict.update(super_dict_entry)
            print(f"Added: {entry}")
            break
            # print(entry, words, super_dict)

In [80]:
print("--------- ITEMS --------------")
for k in super_dict.keys():
    print(k)

--------- ITEMS --------------
IN
WITHIN
OR
GOLD


In [51]:
a = dict()
a.update({'hi':1})

In [77]:
list(a.keys())

['hi']